[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giordamaug/EG-identification---Data-Science-in-App-Springer/blob/main/notebook/EssentialGenes_Karateclub.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giordamaug/EG-identification---Data-Science-in-App-Springer/main?filepath=notebook%2FEssentialGenes_Karateclub.ipynb)

# Loading required libraries

### Install libraries

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
!pip install -q karateclub
if not IN_COLAB:
    !pip install -q pandas
    !pip install -q sklearn
    !pip install -q imblearn
    !pip install -q xgboost
    !pip install -q tqdm

     |████████████████████████████████| 62 kB 622 kB/s 
     |████████████████████████████████| 1.8 MB 16.8 MB/s 
     |████████████████████████████████| 24.1 MB 1.5 MB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 


In [2]:
import warnings
warnings.filterwarnings('ignore')
import random
import numpy as np
import pandas as pd
def set_seed(seed=1):
    random.seed(seed)
    np.random.seed(seed)

# Download dataset from Github

In [3]:
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/ppi.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/labels.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/bio_attributes.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/net_attributes.csv
!wget https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/gtex_attributes.csv

--2022-05-12 18:25:09--  https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/ppi.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3966521 (3.8M) [text/plain]
Saving to: ‘ppi.csv’

ppi.csv             100%[===================>]   3.78M  --.-KB/s    in 0.04s   

2022-05-12 18:25:10 (86.4 MB/s) - ‘ppi.csv’ saved [3966521/3966521]

--2022-05-12 18:25:10--  https://raw.githubusercontent.com/giordamaug/EG-identification---Data-Science-in-App-Springer/main/data/labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent,

# Load the label
Only a subset of genes are selected for classification:
+ genes belonging to CS0 group, that are labeled as Essential (E);
+ genes belonging to CS6, CS7, ..., CS9 groups, that are labeled as Not-Essential (NE).

All remaining genes belong to intermediate groups (CS1-CS5) and are considered undetermined (label ND) 

In [ ]:
labels = pd.read_csv("labels.csv", index_col='name')
labels = labels[labels["CS0_vs_CS6-9"].isin(['E', 'NE']) == True]       # drop any gene with undefined (ND) label
genes = labels.index.values                                             # get genes with defined labels (E or NE)
labels = labels.reset_index()                                           # reindex genes by consecutive integers
labels['index'] = labels.index
gene2idx_mapping = { v[1] : v[0]  for v in labels[['index', 'name']].values }           # create mapping index by gene name
idx2gene_mapping = { v[0] : v[1]  for v in labels[['index', 'name']].values }           # create mapping index by gene name
print(f'Selected {len(genes)} genes')
gene2idx_mapping

## Encode the labels
String labels E and Ne are respectively encoded to 0 and 1.
The array `y` containes numeric labels of genes.

In [33]:
from sklearn import preprocessing
from collections import Counter
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(labels['CS0_vs_CS6-9'].values)  
classes_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(classes_mapping, Counter(y))

{'E': 0, 'NE': 1} Counter({1: 3069, 0: 745})


# Load attributes to be used
We identified three sets of attributes:
1. bio attributes, related to gene information (such as, expression, etc.)
2. net attributes, derived from role of gene/node in the network (such as, degree, centrality, etc.)
3. GTEX-* attribute, additional biological information of genes 
Based on user selection, the node attributes are appended in a single matrix of attributes (`x`)

In the attribute matrix `x` there can be NaN or Infinite values. They are corrected as it follow:
+ NaN is replaced by the mean in the attribute range, 
+ Infinte value is replaced by the maximum in the range.

After Nan and Infinite values fixing, the attributes are normalized with Z-score or MinMax normalization functions.

At the end, only nodes (genes) with E or NE labels are selected for the classification

In [104]:
#@title Choose attributes { form-width: "20%" }
normalize_node = "zscore" #@param ["", "zscore", "minmax"]
bio = True #@param {type:"boolean"}
gtex = True #@param {type:"boolean"}
net = False #@param {type:"boolean"}
variable_name = "bio"
bio_df = pd.read_csv("bio_attributes.csv", index_col='name') if bio else pd.DataFrame()
gtex_df = pd.read_csv("gtex_attributes.csv", index_col='name') if gtex else pd.DataFrame()
net_df = pd.read_csv("net_attributes.csv", index_col='name') if net else pd.DataFrame()
x = pd.concat([bio_df, gtex_df, net_df], axis=1)
print(f'Found {x.isnull().sum().sum()} NaN values and {np.isinf(x).values.sum()} Infinite values')
for col in x.columns[x.isna().any()].tolist():
  mean_value=x[col].mean()          # Replace NaNs in column with the mean of values in the same column
  if mean_value is not np.nan:
    x[col].fillna(value=mean_value, inplace=True)
  else:                             # otherwise, if the mean is NaN, remove the column
    x = x.drop(col, 1)
if normalize_node == 'minmax':
  print("X attributes normalization (minmax)...")
  x = (x-x.min())/(x.max()-x.min())
elif normalize_node == 'zscore':
  print("X attributes normalization (zscore)...")
  x = (x-x.mean())/x.std()
x = x.loc[genes]
x["index"] = x.index
x = x.replace({"index": gene2idx_mapping})
x = x.set_index('index')
print(f'New attribute matrix x{x.shape}')

Found 15705 NaN values and 0 Infinite values
X attributes normalization (zscore)...
New attribute matrix x(3814, 105)


# Load the PPI+MET network
The PPI networks is loaded from a CSV file, where
*   `A` is the column name for edge source (gene name)
*   `B` is the column name for edge target (gene name)
*   `weight` is the column name for edge weight
Only some method use the PPI netoworks, as an example all GCN methods, and Node2Vec.

The PPI+MET network is reduced by removing genes with undetermined labels

In [54]:
import networkx as nx
ppi = pd.read_csv('ppi.csv')                                               # read PPI+MET network from CSV file
ppi = ppi.loc[((ppi['A'].isin(genes)) & (ppi['B'].isin(genes)))]           # reduce network only to selected nodes/genes
ppi = ppi.replace({"A": gene2idx_mapping, "B": gene2idx_mapping})          # replace gene name in ppi

## Load the PPI+MET in Networkx graph

In [82]:
edge_list = [list(v) for v in list(ppi[['A','B', 'weight']].values)]      # get the edge list (with weights)
G = nx.Graph()
G.add_nodes_from(range(len(genes)))                                       # add all nodes (genes, also isolated ones)
G.add_weighted_edges_from(edge_list)                                      # add all edges
print(nx.info(G))
print(f"There are {len(list(nx.isolates(G)))} isolated genes")

Graph with 3814 nodes and 107513 edges
There are 124 isolated genes


# Network embedding with Karateclub

In [83]:
import karateclub as kc
params = {"walk_number": 10, 
          "walk_length": 80, 
          "p": 1.0, 
          "q": 1.0, 
          "dimensions": 128, 
          "workers": 4, 
          "window_size": 5, 
          "epochs": 1, 
          "learning_rate": 0.05, 
          "min_count": 1, 
          "seed": 42}
n2v = kc.Node2Vec(**params)
n2v.fit(G)
embedding = n2v.get_embedding()


array([[ 7.0781210e-03, -6.9289198e-03,  6.2267901e-03, ...,
         6.9546867e-03, -1.3784068e-03,  5.5296873e-03],
       [-3.1469050e-01,  1.4522707e+00,  1.5635647e+00, ...,
         4.6674901e-01,  1.6915725e+00, -1.5065259e-01],
       [-1.0101389e-02,  1.9564308e-02,  3.3200890e-02, ...,
         1.7411543e-02,  3.5782680e-02,  3.9251661e-03],
       ...,
       [ 4.8841890e-03,  2.8180766e-03, -2.6046089e-03, ...,
         4.1015390e-03,  1.0039675e-03,  5.9718890e-03],
       [-2.5160487e-03, -6.7671104e-03, -8.4391382e-04, ...,
         6.5326602e-03,  9.4363414e-04, -1.7533338e-03],
       [ 3.4988266e-03, -2.2516224e-04,  2.2633239e-03, ...,
         7.9664886e-03, -2.3658788e-03,  7.4227876e-03]], dtype=float32)

## Convert embedding to dataframe and append it to attribute matrix

In [106]:
embedding = n2v.get_embedding()
embedding_df = pd.DataFrame(embedding, columns = ['N2V_' + str(i + 1)  for i in range(embedding.shape[1])])
x = pd.concat([embedding_df, x], axis=1)
x

,N2V_1,N2V_2,N2V_3,N2V_4,N2V_5,N2V_6,N2V_7,N2V_8,N2V_9,N2V_10,...,GTEX-1497J-0826-SM-5NQAJ,GTEX-1A3MW-2226-SM-73KUX,GTEX-1K9T9-1826-SM-CXZK2,GTEX-REY6-1826-SM-EAZAT,GTEX-1JMQK-1926-SM-CJI3B,GTEX-QLQW-1626-SM-CMKFE,GTEX-T5JC-1626-SM-EZ6KW,GTEX-RU72-1926-SM-EAZ3F,GTEX-R55E-2026-SM-EZ6L1,GTEX-TKQ2-0626-SM-EZ6LB
0,0.007078,-0.006929,0.006227,-0.004060,-0.004023,-0.000733,-0.005152,-0.000972,-0.004319,-0.002082,...,-0.029364,-0.040208,-0.027543,-0.029748,-0.007489,0.001717,-0.032811,-0.029112,-0.030127,-0.020349
1,-0.314691,1.452271,1.563565,-1.756117,-2.019951,-1.980596,-1.513794,0.737421,0.861948,-0.501536,...,-0.047549,-0.063466,-0.040201,-0.035047,-0.045901,-0.050888,-0.054778,-0.042525,-0.045292,-0.041849
2,-0.010101,0.019564,0.033201,-0.031460,-0.029652,-0.039891,-0.030526,0.014598,0.020625,-0.001931,...,-0.043949,-0.047665,-0.035902,-0.033159,-0.040219,-0.050548,-0.048761,-0.041574,-0.050668,-0.039796
3,-0.015046,0.065999,0.063583,-0.075479,-0.099544,-0.084402,-0.077665,0.027764,0.043757,-0.021771,...,-0.050044,-0.072177,-0.041171,-0.035261,-0.046443,-0.049097,-0.053377,-0.043635,-0.056787,-0.043876
4,-0.013120,0.069363,0.067665,-0.089358,-0.089655,-0.093795,-0.067302,0.032029,0.044709,-0.019946,...,-0.041803,-0.053832,-0.034052,-0.032531,-0.041925,-0.048416,-0.046293,-0.039348,-0.041870,-0.039756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3809,-0.007301,-0.004729,0.000208,-0.000896,-0.002983,-0.001329,-0.000473,-0.006346,0.004187,-0.006409,...,-0.045287,-0.059927,-0.039260,-0.034125,-0.043923,-0.050864,-0.051023,-0.040029,-0.045041,-0.041595
3810,-0.006938,0.002022,0.007521,-0.002969,-0.003886,0.002115,-0.000829,-0.001578,-0.000284,-0.007655,...,-0.051700,-0.078242,-0.042949,-0.035807,-0.050502,-0.058912,-0.056568,-0.044861,-0.057343,-0.046881
3811,0.004884,0.002818,-0.002605,0.003180,0.000103,0.001496,-0.005055,0.001928,-0.004809,0.005321,...,-0.051932,-0.078712,-0.043129,-0.035928,-0.050612,-0.059016,-0.057277,-0.044986,-0.057999,-0.047053
3812,-0.002516,-0.006767,-0.000844,-0.006279,-0.004631,-0.003495,-0.004777,0.004605,-0.002937,0.005428,...,-0.047313,-0.068020,-0.039901,-0.034707,-0.046296,-0.051932,-0.050475,-0.043061,-0.050862,-0.043132


# k-fold cross validation

### Validate

In [107]:
#@title Choose classifier { run: "auto", form-width: "20%" }
method = "XGB" #@param ["SVM", "XGB", "RF", "MLP", "RUS", "DUMMY"]
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector as selector
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.neural_network import MLPClassifier
import sys
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from tqdm import tqdm
from sklearn.metrics import *
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import RUSBoostClassifier
from sklearn.dummy import DummyClassifier
seed=1
set_seed(seed)
nfolds = 5
kf = KFold(n_splits=nfolds)
accuracies, mccs = [], []
cm = [[0,0],[0,0]]
X = x.to_numpy()

if method == 'SVM':
  clf = SVC(random_state=seed, probability=True)
elif method == 'MLP':
  clf = MLPClassifier(random_state=seed)
elif method == 'RF':
  clf = RandomForestClassifier(random_state=seed)
elif method == 'XGB':
  clf = XGBClassifier(random_state=seed)
elif method == 'RUS':
  clf = RUSBoostClassifier(random_state=seed)
else:
  raise ValueError("Method not implemented!")

cma = np.array([[0,0],[0,0]])
mm = np.array([], dtype=np.int)
predictions = np.array([])
columns_names = ["Accuracy","BA", "Sensitivity", "Specificity","MCC", 'CM']
scores = pd.DataFrame(columns=columns_names)
for fold, (train_idx, test_idx) in enumerate(tqdm(kf.split(np.arange(len(X))), total=kf.get_n_splits(), desc=f"{nfolds}-fold")):
    train_x, train_y, test_x, test_y = X[train_idx], y[train_idx], X[test_idx], y[test_idx],
    mm = np.concatenate((mm, test_idx))
    probs = clf.fit(train_x, train_y).predict(test_x)
    preds = (probs > 0.5) * 1
    cm = confusion_matrix(test_y, preds)
    cma += cm
    predictions = np.concatenate((predictions, preds))
    scores = scores.append(pd.DataFrame([[accuracy_score(test_y, preds), balanced_accuracy_score(test_y, preds), 
        cm[0,0]/(cm[0,0]+cm[0,1]), cm[1,1]/(cm[1,0]+cm[1,1]), 
        matthews_corrcoef(test_y, preds), cm]], columns=columns_names, index=[fold]))
df_scores = pd.DataFrame(scores.mean(axis=0)).T
df_scores.index=[f'{method}']
df_scores['CM'] = [cma]
df_scores

5-fold: 100%|██████████| 5/5 [00:19<00:00,  3.85s/it]


,Accuracy,BA,Sensitivity,Specificity,MCC,CM
XGB,0.888047,0.775222,0.590393,0.96005,0.616606,"[[440, 305], [122, 2947]]"


# Print predictions

In [108]:
p = np.zeros(len(y))
p[mm] = predictions
labels['predictions'] = ['NE' if x>0 else 'E' for x in p]
labels

,name,CS0_vs_CS6-9,index,predictions
0,ENSG00000001036,NE,0,NE
1,ENSG00000001461,NE,1,NE
2,ENSG00000001561,NE,2,NE
3,ENSG00000001630,NE,3,E
4,ENSG00000001631,NE,4,NE
...,...,...,...,...
3809,ENSG00000288283,NE,3809,NE
3810,ENSG00000288359,NE,3810,NE
3811,ENSG00000288407,NE,3811,NE
3812,ENSG00000288478,NE,3812,NE
